In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import f1_score
import os
import glob

%matplotlib inline

## MODEL

In [14]:
def f1_comp(cf):
    prec = 0
    rec = 0
    f1 = 0
    if (cf[0] + cf[2]) != 0:
        prec = cf[0] / (cf[0] + cf[2])
    if (cf[0] + cf[3]) != 0:
        rec  = cf[0] / (cf[0] + cf[3])
    if (prec + rec) != 0:
        f1   = 2 * (prec * rec / (prec+rec))
        
    return f1, prec, rec

In [4]:
def conf_mat(pred,truth):
    res = [0,0,0,0]
    a = 0
    
    for i in range(len(truth)):
        if truth[i] == 1:
            if truth[i] == pred[i]:
                a = 0
            else:
                a = 2
                
        else:
            if truth[i] == pred[i]:
                a = 1
            else:
                a = 3
                
        res[a] = res[a] + 1
     
    print(res)
    return res

In [5]:
def predict(r_mean,limit):
    res = [0 for i in range(len(r_mean))]

    for i in range(1,len(r_mean)):
        if abs(r_mean[i] - r_mean[i-1]) > limit:
            res[i-1] = 1
            res[i] = 1
            if i < (len(r_mean)-1):
                res[i+1] = 1
        
    return res

In [89]:
def compute_rolling(arr, kernel_a, kernel_b,limit):
    start = kernel_a + kernel_b
    pred = np.zeros(len(arr))
    pred_std = np.zeros(len(arr))
    pred_normal = np.zeros(len(arr))
    
    
    for i in range(start,len(arr)):
        a_sset = arr[(i-start):(i-kernel_b)]
        b_sset = arr[(i-kernel_b):i]
        
        a_mu = np.mean(a_sset)
        a_std = np.std(a_sset)
        
        b_mu = np.mean(b_sset)
        b_std = np.std(b_sset)
        
#         print(a_sset,b_sset,arr[i])
        
        
#         if abs(b_mu-a_mu) >= limit * abs(a_std):
#             pred[i] = 1
            
        if abs(b_std-a_std) >= limit * abs(a_std):
            pred_std[i-1] = 1
        
#         if arr[i] > (b_mu + limit * b_std) or arr[i] < (b_mu - limit * b_std):
#             pred_normal[i] = 1
            
            
    return pred, pred_std, pred_normal    

In [87]:
z = [i for i in range(20)]
compute_rolling(z,5,5,0.5)

[0, 1, 2, 3, 4] [5, 6, 7, 8, 9] 10
[1, 2, 3, 4, 5] [6, 7, 8, 9, 10] 11
[2, 3, 4, 5, 6] [7, 8, 9, 10, 11] 12
[3, 4, 5, 6, 7] [8, 9, 10, 11, 12] 13
[4, 5, 6, 7, 8] [9, 10, 11, 12, 13] 14
[5, 6, 7, 8, 9] [10, 11, 12, 13, 14] 15
[6, 7, 8, 9, 10] [11, 12, 13, 14, 15] 16
[7, 8, 9, 10, 11] [12, 13, 14, 15, 16] 17
[8, 9, 10, 11, 12] [13, 14, 15, 16, 17] 18
[9, 10, 11, 12, 13] [14, 15, 16, 17, 18] 19


(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]))

## IMPLEMENTATION

In [90]:
input_dir = './../../train/KPI/'

# summary = pd.DataFrame(columns=['KPI', 'TP', 'TN', 'FP', 'FN', 'PRECISION', 'RECALL', 'F1_SCORE'])
summary_2 = pd.DataFrame(columns=['KPI', 'TP', 'TN', 'FP', 'FN', 'PRECISION', 'RECALL', 'F1_SCORE'])
# summary_3 = pd.DataFrame(columns=['KPI', 'TP', 'TN', 'FP', 'FN', 'PRECISION', 'RECALL', 'F1_SCORE'])


for fname in os.listdir(input_dir):
    limit = 1
    
    high_1 = 0
    high_2 = 0
    high_3 = 0
    
    while limit <= 12:
    
        df  = pd.read_csv(os.path.join(input_dir, fname), index_col='timestamp')
        kpi_name = df['KPI ID'].values[0]
        print(kpi_name)
        df = df.drop(['KPI ID'], axis=1)


        # Make Predictions
        y_pred, y_std_pred, y_normal_pred = compute_rolling(df.value.values,5,5,limit)

        # Generate Confusion Matrix
#         cf = conf_mat(y_pred,df.label.values)
        cf_2 = conf_mat(y_std_pred,df.label.values)
#         cf_3 = conf_mat(y_normal_pred,df.label.values)


        # F1-Score
#         f1, prec, rec = f1_comp(cf)
        f1_2, prec_2, rec_2 = f1_comp(cf_2)
#         f1_3, prec_3, rec_3 = f1_comp(cf_3)


#         if f1 > high_1:
#             res_1 = [ cf[0], cf[1], cf[2], cf[3], prec, rec, f1,limit ]
#             high_1 = f1
            
        if f1_2 > high_2:
            res_2 = [ cf[0], cf[1], cf[2], cf[3], prec_2, rec_2, f1_2,limit ]
            high_2 = f1_2
            
#         if f1_3 > high_3:
#             res_3 = [ cf[0], cf[1], cf[2], cf[3], prec_3, rec_3, f1_3,limit ]
#             high_3 = f1_3
        
        limit += 0.1
        
        
#     summary = summary.append({'KPI': kpi_name, 
#                               'LIMIT': res_1[7],
#                                'TP': res_1[0],  
#                                'TN': res_1[1], 
#                                'FP': res_1[2], 
#                                'FN': res_1[3], 
#                                'PRECISION': res_1[4], 
#                                'RECALL': res_1[5],     
#                                'F1_SCORE': res_1[6] }, ignore_index=True)

    summary_2 = summary_2.append({'KPI': kpi_name, 
                                  'LIMIT': res_2[7],
                                   'TP': res_2[0],  
                                   'TN': res_2[1], 
                                   'FP': res_2[2], 
                                   'FN': res_2[3], 
                                   'PRECISION': res_2[4], 
                                   'RECALL': res_2[5],     
                                   'F1_SCORE': res_2[6] }, ignore_index=True)

#     summary_3 = summary_3.append({'KPI': kpi_name, 
#                                   'LIMIT': res_3[7],
#                                    'TP': res_3[0],  
#                                    'TN': res_3[1], 
#                                    'FP': res_3[2], 
#                                    'FN': res_3[3], 
#                                    'PRECISION': res_3[4], 
#                                    'RECALL': res_3[5],     
#                                    'F1_SCORE': res_3[6] }, ignore_index=True)

    
    

a40b1df87e3f1c87
[269, 121403, 112, 16141]
a40b1df87e3f1c87
[268, 123337, 113, 14207]
a40b1df87e3f1c87
[265, 125053, 116, 12491]
a40b1df87e3f1c87
[261, 126574, 120, 10970]
a40b1df87e3f1c87
[259, 127859, 122, 9685]
a40b1df87e3f1c87
[254, 128946, 127, 8598]
a40b1df87e3f1c87
[253, 129902, 128, 7642]
a40b1df87e3f1c87
[252, 130703, 129, 6841]
a40b1df87e3f1c87
[251, 131515, 130, 6029]
a40b1df87e3f1c87
[248, 132169, 133, 5375]
a40b1df87e3f1c87
[248, 132698, 133, 4846]
a40b1df87e3f1c87
[246, 133183, 135, 4361]
a40b1df87e3f1c87
[245, 133555, 136, 3989]
a40b1df87e3f1c87
[242, 133930, 139, 3614]
a40b1df87e3f1c87
[238, 134294, 143, 3250]
a40b1df87e3f1c87
[238, 134587, 143, 2957]
a40b1df87e3f1c87
[238, 134836, 143, 2708]
a40b1df87e3f1c87
[238, 135062, 143, 2482]
a40b1df87e3f1c87
[238, 135270, 143, 2274]
a40b1df87e3f1c87
[237, 135437, 144, 2107]
a40b1df87e3f1c87
[237, 135612, 144, 1932]
a40b1df87e3f1c87
[237, 135746, 144, 1798]
a40b1df87e3f1c87
[237, 135879, 144, 1665]
a40b1df87e3f1c87
[237, 136018,

[0, 8233, 12, 2]
b3b2e6d1a791d63a
[0, 8233, 12, 2]
b3b2e6d1a791d63a
[0, 8233, 12, 2]
b3b2e6d1a791d63a
[0, 8233, 12, 2]
b3b2e6d1a791d63a
[0, 8233, 12, 2]
b3b2e6d1a791d63a
[0, 8233, 12, 2]
da403e4e3f87c9e0
[6962, 86886, 704, 34483]
da403e4e3f87c9e0
[6673, 97722, 993, 23647]
da403e4e3f87c9e0
[6667, 98427, 999, 22942]
da403e4e3f87c9e0
[6661, 99343, 1005, 22026]
da403e4e3f87c9e0
[6652, 100277, 1014, 21092]
da403e4e3f87c9e0
[6644, 101105, 1022, 20264]
da403e4e3f87c9e0
[6635, 101797, 1031, 19572]
da403e4e3f87c9e0
[6630, 101945, 1036, 19424]
da403e4e3f87c9e0
[6617, 102597, 1049, 18772]
da403e4e3f87c9e0
[6608, 102805, 1058, 18564]
da403e4e3f87c9e0
[6605, 103422, 1061, 17947]
da403e4e3f87c9e0
[6601, 103737, 1065, 17632]
da403e4e3f87c9e0
[6599, 103889, 1067, 17480]
da403e4e3f87c9e0
[6596, 104037, 1070, 17332]
da403e4e3f87c9e0
[6594, 104354, 1072, 17015]
da403e4e3f87c9e0
[6591, 104441, 1075, 16928]
da403e4e3f87c9e0
[6585, 104593, 1081, 16776]
da403e4e3f87c9e0
[6579, 104725, 1087, 16644]
da403e4e3f

[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[46, 8441, 34, 263]
046ec29ddf80d62e
[

[29, 127347, 209, 1386]
09513ae3e75778a3
[29, 127362, 209, 1371]
09513ae3e75778a3
[28, 127382, 210, 1351]
09513ae3e75778a3
[27, 127399, 211, 1334]
09513ae3e75778a3
[25, 127417, 213, 1316]
09513ae3e75778a3
[25, 127430, 213, 1303]
09513ae3e75778a3
[25, 127443, 213, 1290]
09513ae3e75778a3
[25, 127456, 213, 1277]
09513ae3e75778a3
[25, 127470, 213, 1263]
09513ae3e75778a3
[25, 127494, 213, 1239]
09513ae3e75778a3
[25, 127507, 213, 1226]
09513ae3e75778a3
[24, 127517, 214, 1216]
09513ae3e75778a3
[23, 127529, 215, 1204]
09513ae3e75778a3
[23, 127541, 215, 1192]
09513ae3e75778a3
[23, 127557, 215, 1176]
09513ae3e75778a3
[23, 127568, 215, 1165]
09513ae3e75778a3
[21, 127588, 217, 1145]
09513ae3e75778a3
[21, 127599, 217, 1134]
09513ae3e75778a3
[21, 127609, 217, 1124]
09513ae3e75778a3
[21, 127618, 217, 1115]
09513ae3e75778a3
[21, 127636, 217, 1097]
09513ae3e75778a3
[20, 127647, 218, 1086]
09513ae3e75778a3
[20, 127658, 218, 1075]
09513ae3e75778a3
[19, 127665, 219, 1068]
09513ae3e75778a3
[19, 127674, 219

[192, 146046, 895, 496]
cff6d3c01e6a6bfa
[191, 146062, 896, 480]
cff6d3c01e6a6bfa
[189, 146082, 898, 460]
cff6d3c01e6a6bfa
[188, 146096, 899, 446]
cff6d3c01e6a6bfa
[186, 146113, 901, 429]
cff6d3c01e6a6bfa
[183, 146130, 904, 412]
cff6d3c01e6a6bfa
[180, 146149, 907, 393]
cff6d3c01e6a6bfa
[178, 146168, 909, 374]
cff6d3c01e6a6bfa
[178, 146185, 909, 357]
cff6d3c01e6a6bfa
[178, 146197, 909, 345]
cff6d3c01e6a6bfa
[177, 146204, 910, 338]
cff6d3c01e6a6bfa
[177, 146215, 910, 327]
cff6d3c01e6a6bfa
[177, 146228, 910, 314]
cff6d3c01e6a6bfa
[176, 146240, 911, 302]
cff6d3c01e6a6bfa
[172, 146246, 915, 296]
cff6d3c01e6a6bfa
[170, 146255, 917, 287]
cff6d3c01e6a6bfa
[168, 146267, 919, 275]
cff6d3c01e6a6bfa
[165, 146277, 922, 265]
cff6d3c01e6a6bfa
[164, 146289, 923, 253]
cff6d3c01e6a6bfa
[163, 146293, 924, 249]
cff6d3c01e6a6bfa
[162, 146304, 925, 238]
cff6d3c01e6a6bfa
[161, 146310, 926, 232]
cff6d3c01e6a6bfa
[159, 146317, 928, 225]
cff6d3c01e6a6bfa
[158, 146329, 929, 213]
cff6d3c01e6a6bfa
[155, 146335, 93

...

[0, 128176, 143, 294]
9bd90500bfd11edb
[0, 128182, 143, 288]
9bd90500bfd11edb
[0, 128191, 143, 279]
9bd90500bfd11edb
[0, 128197, 143, 273]
9bd90500bfd11edb
[0, 128209, 143, 261]
9bd90500bfd11edb
[0, 128218, 143, 252]
9bd90500bfd11edb
[0, 128223, 143, 247]
9bd90500bfd11edb
[0, 128228, 143, 242]
9bd90500bfd11edb
[0, 128235, 143, 235]
9bd90500bfd11edb
[0, 128242, 143, 228]
9bd90500bfd11edb
[0, 128250, 143, 220]
9bd90500bfd11edb
[0, 128253, 143, 217]
9bd90500bfd11edb
[0, 128256, 143, 214]
9bd90500bfd11edb
[0, 128263, 143, 207]
9bd90500bfd11edb
[0, 128267, 143, 203]
9bd90500bfd11edb
[0, 128272, 143, 198]
9bd90500bfd11edb
[0, 128276, 143, 194]
e0770391decc44ce
[439, 121271, 2635, 22679]
e0770391decc44ce
[390, 123181, 2684, 20769]
e0770391decc44ce
[352, 124815, 2722, 19135]
e0770391decc44ce
[320, 126360, 2754, 17590]
e0770391decc44ce
[292, 127657, 2782, 16293]
e0770391decc44ce
[267, 128828, 2807, 15122]
e0770391decc44ce
[243, 129940, 2831, 14010]
e0770391decc44ce
[233, 130825, 2841, 13125]
e0

[3, 8681, 1, 99]
8a20c229e9860d0c
[3, 8683, 1, 97]
8a20c229e9860d0c
[3, 8684, 1, 96]
8a20c229e9860d0c
[3, 8685, 1, 95]
8a20c229e9860d0c
[3, 8688, 1, 92]
8a20c229e9860d0c
[3, 8689, 1, 91]
8a20c229e9860d0c
[3, 8693, 1, 87]
8a20c229e9860d0c
[3, 8694, 1, 86]
8a20c229e9860d0c
[3, 8697, 1, 83]
8a20c229e9860d0c
[3, 8699, 1, 81]
8a20c229e9860d0c
[3, 8701, 1, 79]
8a20c229e9860d0c
[3, 8703, 1, 77]
8a20c229e9860d0c
[3, 8703, 1, 77]
8a20c229e9860d0c
[3, 8705, 1, 75]
8a20c229e9860d0c
[3, 8706, 1, 74]
8a20c229e9860d0c
[3, 8708, 1, 72]
8a20c229e9860d0c
[3, 8710, 1, 70]
8a20c229e9860d0c
[3, 8712, 1, 68]
8a20c229e9860d0c
[3, 8714, 1, 66]
8a20c229e9860d0c
[3, 8714, 1, 66]
8a20c229e9860d0c
[3, 8714, 1, 66]
8a20c229e9860d0c
[3, 8716, 1, 64]
8a20c229e9860d0c
[3, 8716, 1, 64]
8a20c229e9860d0c
[3, 8717, 1, 63]
8a20c229e9860d0c
[3, 8718, 1, 62]
8a20c229e9860d0c
[3, 8720, 1, 60]
8a20c229e9860d0c
[3, 8721, 1, 59]
8a20c229e9860d0c
[3, 8722, 1, 58]
8a20c229e9860d0c
[3, 8726, 1, 54]
769894baefea4e9e
[5, 7727, 4, 1

[79, 119212, 9502, 60]
1c35dbf57f55f5e4
[79, 119213, 9502, 59]
1c35dbf57f55f5e4
[78, 119216, 9503, 56]
1c35dbf57f55f5e4
[77, 119218, 9504, 54]
1c35dbf57f55f5e4
[76, 119219, 9505, 53]
1c35dbf57f55f5e4
[75, 119222, 9506, 50]
1c35dbf57f55f5e4
[73, 119224, 9508, 48]
1c35dbf57f55f5e4
[72, 119225, 9509, 47]
1c35dbf57f55f5e4
[71, 119227, 9510, 45]
1c35dbf57f55f5e4
[71, 119228, 9510, 44]
1c35dbf57f55f5e4
[71, 119229, 9510, 43]
1c35dbf57f55f5e4
[71, 119230, 9510, 42]
1c35dbf57f55f5e4
[70, 119234, 9511, 38]
1c35dbf57f55f5e4
[69, 119234, 9512, 38]
1c35dbf57f55f5e4
[68, 119234, 9513, 38]
1c35dbf57f55f5e4
[68, 119235, 9513, 37]
1c35dbf57f55f5e4
[68, 119237, 9513, 35]
1c35dbf57f55f5e4
[68, 119239, 9513, 33]
1c35dbf57f55f5e4
[66, 119239, 9515, 33]
1c35dbf57f55f5e4
[66, 119239, 9515, 33]
1c35dbf57f55f5e4
[66, 119239, 9515, 33]
1c35dbf57f55f5e4
[65, 119241, 9516, 31]
1c35dbf57f55f5e4
[61, 119241, 9520, 31]
1c35dbf57f55f5e4
[61, 119241, 9520, 31]
1c35dbf57f55f5e4
[61, 119242, 9520, 30]
1c35dbf57f55f5e4


[139, 146261, 974, 294]
71595dd7171f4540
[139, 146269, 974, 286]
71595dd7171f4540
[137, 146274, 976, 281]
71595dd7171f4540
[136, 146280, 977, 275]
71595dd7171f4540
[135, 146296, 978, 259]
71595dd7171f4540
[135, 146301, 978, 254]
71595dd7171f4540
[135, 146312, 978, 243]
71595dd7171f4540
[134, 146318, 979, 237]
71595dd7171f4540
[132, 146327, 981, 228]
71595dd7171f4540
[131, 146337, 982, 218]
71595dd7171f4540
[129, 146349, 984, 206]
71595dd7171f4540
[128, 146355, 985, 200]
71595dd7171f4540
[127, 146359, 986, 196]
71595dd7171f4540
[125, 146367, 988, 188]
71595dd7171f4540
[119, 146374, 994, 181]
71595dd7171f4540
[119, 146379, 994, 176]
71595dd7171f4540
[118, 146387, 995, 168]
71595dd7171f4540
[117, 146390, 996, 165]
71595dd7171f4540
[113, 146396, 1000, 159]
71595dd7171f4540
[113, 146401, 1000, 154]
71595dd7171f4540
[111, 146408, 1002, 147]
71595dd7171f4540
[110, 146414, 1003, 141]
71595dd7171f4540
[107, 146416, 1006, 139]
71595dd7171f4540
[105, 146418, 1008, 137]
71595dd7171f4540
[103, 1464

[23, 15465, 6, 113185]
a5bf5d65261d859a
[23, 15466, 6, 113184]
a5bf5d65261d859a
[23, 15469, 6, 113181]
a5bf5d65261d859a
[23, 15472, 6, 113178]
a5bf5d65261d859a
[23, 15472, 6, 113178]
a5bf5d65261d859a
[23, 15473, 6, 113177]
a5bf5d65261d859a
[23, 15473, 6, 113177]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a
[23, 15474, 6, 113176]
a5bf5d65261d859a


In [92]:
summary_2.to_csv('RollStd_A_B.csv')

In [69]:
summary.to_csv('Rolling_STD_1.csv')

summary_2.to_csv('Rolling_STD_2.csv')

summary_3.to_csv('Rolling_STD_3.csv')

In [28]:
summary

,KPI,TP,TN,FP,FN,PRECISION,RECALL,F1_SCORE
0,a40b1df87e3f1c87,145,128731,236,8813,0.380577,0.016187,0.031053
1,b3b2e6d1a791d63a,4,7743,8,492,0.333333,0.008065,0.015748
2,da403e4e3f87c9e0,6736,103135,930,18234,0.878685,0.269764,0.412796
3,046ec29ddf80d62e,55,7463,25,1241,0.687500,0.042438,0.079942
4,18fbb1d5a5dc099d,503,118866,7360,2399,0.063970,0.173329,0.093451
5,09513ae3e75778a3,40,123539,198,5194,0.168067,0.007642,0.014620
6,07927a9a18fa19ae,71,2878,59,7952,0.546154,0.008850,0.017417
7,cff6d3c01e6a6bfa,196,139672,891,6870,0.180313,0.027738,0.048080
8,54e8a140f6237526,0,7598,4,646,0.000000,0.000000,0.000000
9,c58bfcbacb2822d1,55,78386,69,50157,0.443548,0.001095,0.002185


In [30]:
summary_2

,KPI,TP,TN,FP,FN,PRECISION,RECALL,F1_SCORE
0,a40b1df87e3f1c87,141,118935,240,18609,0.370079,0.007520,0.014740
1,b3b2e6d1a791d63a,2,7196,10,1039,0.166667,0.001921,0.003799
2,da403e4e3f87c9e0,546,108204,7120,13165,0.071224,0.039822,0.051083
3,046ec29ddf80d62e,9,7355,71,1349,0.112500,0.006627,0.012517
4,18fbb1d5a5dc099d,957,107565,6906,13700,0.121709,0.065293,0.084991
5,09513ae3e75778a3,44,116407,194,12326,0.184874,0.003557,0.006980
6,07927a9a18fa19ae,37,9367,93,1463,0.284615,0.024667,0.045399
7,cff6d3c01e6a6bfa,247,127033,840,19509,0.227231,0.012503,0.023701
8,54e8a140f6237526,2,7092,2,1152,0.500000,0.001733,0.003454
9,c58bfcbacb2822d1,43,100012,81,28531,0.346774,0.001505,0.002997


In [29]:
summary_2.to_csv('Rolling_Mean_5_set.csv')

summary_2.to_csv('Rolling_Mean_5.csv')

## Single Model

In [63]:
input_dir = './../../train/KPI/'
fname = 'train_046ec29ddf80d62e.csv'

test = pd.DataFrame(columns=['KPI', 'TP', 'TN', 'FP', 'FN', 'PRECISION', 'RECALL', 'F1_SCORE'])

limit = 1.0

while limit < 18:


    df  = pd.read_csv(os.path.join(input_dir, fname), index_col='timestamp')
    kpi_name = df['KPI ID'].values[0]
    print(kpi_name)
    df = df.drop(['KPI ID'], axis=1)


    # Make Predictions
    y_pred, y_std_pred, y_normal_pred = compute_rolling(df.value.values,5,5,limit)

    # Generate Confusion Matrix
    cf = conf_mat(y_pred,df.label.values)
    cf_2 = conf_mat(y_normal_pred,df.label.values)
    cf_3 = conf_mat(y_std_pred,df.label.values)
    

    # F1-Score
    f1, prec, rec = f1_comp(cf)
    f1_2, prec_2, rec_2 = f1_comp(cf_2)
    f1_3, prec_3, rec_3 = f1_comp(cf_3)
    

    print(limit,f1,f1_2,f1_3)

    test = test.append({'KPI': kpi_name,
                              'STD': limit,
                                   'TP': cf[0],  
                                   'TN': cf[1], 
                                   'FP': cf[2], 
                                   'FN': cf[3], 
                                   'PRECISION': prec, 
                                   'RECALL': rec,     
                                   'F1_SCORE': f1 }, ignore_index=True)
    
    limit += 0.1
    limit = round(limit,1)

046ec29ddf80d62e
[62, 3439, 18, 5265]
[8, 7873, 72, 831]
[71, 6711, 9, 1993]
1.0 0.022933234695764748 0.017410228509249184 0.06623134328358209
046ec29ddf80d62e
[61, 3826, 19, 4878]
[8, 7873, 72, 831]
[59, 7342, 21, 1362]
1.1 0.024307631002191674 0.017410228509249184 0.07861425716189208
046ec29ddf80d62e
[60, 3977, 20, 4727]
[8, 7873, 72, 831]
[59, 7387, 21, 1317]
1.2 0.02465584549003493 0.017410228509249184 0.08104395604395603
046ec29ddf80d62e
[60, 4432, 20, 4272]
[8, 7873, 72, 831]
[59, 7510, 21, 1194]
1.3 0.027198549410698096 0.017410228509249184 0.08852213053263316
046ec29ddf80d62e
[60, 4681, 20, 4023]
[8, 7873, 72, 831]
[59, 7594, 21, 1110]
1.4 0.028825366322363676 0.017410228509249184 0.0944755804643715
046ec29ddf80d62e
[59, 4834, 21, 3870]
[8, 7873, 72, 831]
[58, 7707, 22, 997]
1.5 0.02943377400848092 0.017410228509249184 0.10220264317180616
046ec29ddf80d62e
[58, 5146, 22, 3558]
[8, 7873, 72, 831]
[57, 7829, 23, 875]
1.6 0.03138528138528139 0.017410228509249184 0.11264822134387352

[47, 8387, 33, 317]
[8, 7873, 72, 831]
[48, 8310, 32, 394]
6.8 0.21171171171171174 0.017410228509249184 0.18390804597701152
046ec29ddf80d62e
[47, 8387, 33, 317]
[8, 7873, 72, 831]
[48, 8310, 32, 394]
6.9 0.21171171171171174 0.017410228509249184 0.18390804597701152
046ec29ddf80d62e
[47, 8397, 33, 307]
[8, 7873, 72, 831]
[48, 8310, 32, 394]
7.0 0.21658986175115208 0.017410228509249184 0.18390804597701152
046ec29ddf80d62e
[47, 8404, 33, 300]
[8, 7873, 72, 831]
[48, 8310, 32, 394]
7.1 0.2201405152224824 0.017410228509249184 0.18390804597701152
046ec29ddf80d62e
[47, 8404, 33, 300]
[8, 7873, 72, 831]
[48, 8310, 32, 394]
7.2 0.2201405152224824 0.017410228509249184 0.18390804597701152
046ec29ddf80d62e
[46, 8404, 34, 300]
[8, 7873, 72, 831]
[48, 8310, 32, 394]
7.3 0.21596244131455397 0.017410228509249184 0.18390804597701152
046ec29ddf80d62e
[46, 8410, 34, 294]
[8, 7873, 72, 831]
[48, 8310, 32, 394]
7.4 0.21904761904761905 0.017410228509249184 0.18390804597701152
046ec29ddf80d62e
[46, 8411, 34, 

KeyboardInterrupt: 

In [22]:
y_normal_pred

1

In [55]:
test[test.index == 91]['F1_SCORE']

91    0.233503
Name: F1_SCORE, dtype: float64